In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pint
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr
from dask.distributed import Client
from pint import UnitStrippedWarning
from seapopym.configuration.no_transport.parameter import ForcingParameters
from seapopym.configuration.parameters.parameter_forcing import ForcingUnit
from seapopym.standard.units import StandardUnitsLabels

from seapopym_optimization import Observation, constraint, wrapper
from seapopym_optimization.cost_function import NoTransportCostFunction
from seapopym_optimization.functional_groups import FunctionalGroupOptimizeNoTransport, Parameter
from seapopym_optimization.genetic_algorithm import GeneticAlgorithm, GeneticAlgorithmParameters
from seapopym_optimization.taylor_diagram import ModTaylorDiagram, generate_mod_taylor_diagram

warnings.simplefilter("ignore", category=UnitStrippedWarning)
xr.set_options(
    display_expand_attrs=False,
    display_expand_data_vars=False,
    display_expand_coords=False,
    display_expand_data=False,
)

In [2]:
path_to_forcing = "./forcing.zarr"
path_to_obs_hot = "../../../1_data_processing/1_1_Forcing/products/Hot_obs_zoo_climato_monthly_2002_2015.zarr"
path_to_obs_barents = "../../../1_data_processing/1_1_Forcing/1_optional_extract_barents_sea/zoo_2000_2017.zarr"
export_file_name = "SeapoPym_climato_hot_barents_2_groups"

In [3]:
TIME_START = "2005-01-01"
TIME_END = "2010-01-01"
LATITUDE = 22.75
LONGITUDE = -158
STABILIZATION_DAYS = 90
SAVE = False

## Loading


### Forcing


In [4]:
forcing = xr.open_zarr(path_to_forcing)
forcing = forcing.sel(time=slice(TIME_START, TIME_END))
forcing["temperature"].attrs["units"] = StandardUnitsLabels.temperature.units
forcing.load()

<xarray.Dataset> Size: 424kB
Dimensions:              (depth: 3, latitude: 2, longitude: 2, time: 1827)
Coordinates: (4)
Data variables: (3)

### Epipelagic layer


In [5]:
epi_layer_depth = forcing["pelagic_layer_depth"].sel(depth=0).load()
epi_layer_depth = epi_layer_depth.resample(time="1D").mean()
epi_layer_depth.attrs["units"] = "meter"
epi_layer_depth = epi_layer_depth.pint.quantify()
epi_layer_depth

Magnitude,[[[114.84210764968468 nan] [nan 154.8514838879948]] [[113.10354321928514 nan] [nan 155.16812711459391]] [[114.18750083099961 nan] [nan 155.4294606990115]] ... [[113.9499935505495 nan] [nan 152.39512719007502]] [[113.55127089450652 nan] [nan 152.72394211786622]] [[114.84210764968468 nan] [nan 153.024833731158]]]
Units,meter


<!-- ## Observed NPP -->


### Observations


In [6]:
observations_hot = xr.open_zarr(path_to_obs_hot).load()
observations_hot = observations_hot.resample(time="1D").interpolate("linear").dropna("time")
observations_hot = observations_hot.sel(time=slice(TIME_START, TIME_END))
observations_hot = observations_hot.pint.quantify().pint.to("mg/m^3")
observations_hot = observations_hot * epi_layer_depth
observations_hot = observations_hot.drop_vars("depth")
observations_hot = observations_hot[["day", "night"]]
observations_hot

Magnitude,[[[[175.0393382017864]]] [[[171.91155568709888]]] [[[173.07662716509964]]] ... [[[174.86090174350818]]] [[[173.55167845591396]]] [[[175.0393382017864]]]]
Units,milligram/meter2
Magnitude,[[[[296.67369635689136]]] [[[290.87722330653037]]] [[[292.3472049493092]]] ... [[[297.288307051414]]] [[[294.6494292157888]]] [[[296.67369635689136]]]]
Units,milligram/meter2


In [7]:
observations_barents = xr.open_zarr(path_to_obs_barents).load()
observations_barents = observations_barents.resample(time="1D").interpolate("linear").dropna("time")
observations_barents = observations_barents.sel(time=slice(TIME_START, TIME_END))
observations_barents = observations_barents.pint.quantify().pint.to("mg/m^2").pint.dequantify()
observations_barents

<xarray.Dataset> Size: 29kB
Dimensions:    (time: 1827, latitude: 1, longitude: 1, layer: 1)
Coordinates: (4)
Data variables: (1)

Remove the X first months to let the model reach the stationary state.


In [8]:
decay = pd.Timestamp(TIME_START) + pd.Timedelta(days=STABILIZATION_DAYS)

observations_hot = observations_hot.sel(time=slice(str(decay), None))
observations_barents = observations_barents.sel(time=slice(str(decay), None))

Create structure for SeapoPym simulation.


In [9]:
forcing_parameters = ForcingParameters(
    temperature=ForcingUnit(forcing=forcing["temperature"], resolution=1 / 12, timestep=1),
    primary_production=ForcingUnit(forcing=forcing["primary_production"], resolution=1 / 12, timestep=1),
)

2025-01-31 12:13:36,986 :: Seapodym ::  WARNING ::
|	primary_production unit is milligram / day / meter ** 2, it will be converted to kilogram / day / meter ** 2.



## Setup the parameters and the cost function


In [10]:
functional_groups = [
    FunctionalGroupOptimizeNoTransport(
        name="D1N1",
        day_layer=0,
        night_layer=0,
        energy_coefficient=Parameter("D1N1_energy_coefficient", 0.001, 0.5),
        tr_rate=Parameter("D1N1_tr_rate", -0.3, -0.001),
        tr_max=Parameter("D1N1_tr_max", 0, 100),
        inv_lambda_rate=Parameter("D1N1_inv_lambda_rate", -0.3, -0.001),
        inv_lambda_max=Parameter("D1N1_inv_lambda_max", 100, 200),
    ),
    FunctionalGroupOptimizeNoTransport(
        name="D2N1",
        day_layer=1,
        night_layer=0,
        energy_coefficient=Parameter("D2N1_energy_coefficient", 0.001, 0.5),
        tr_rate=Parameter("D2N1_tr_rate", -0.3, -0.001),
        tr_max=Parameter("D2N1_tr_max", 0, 100),
        inv_lambda_rate=Parameter("D2N1_inv_lambda_rate", -0.3, -0.001),
        inv_lambda_max=Parameter("D2N1_inv_lambda_max", 100, 200),
    ),
]

In [11]:
cost_function = NoTransportCostFunction(
    functional_groups=functional_groups,
    forcing_parameters=forcing_parameters,
    observations=[Observation(observations_hot), Observation(observations_barents)],
)

Set the genetic algorithm meta parameters.


In [12]:
genetic_algo_parameters = GeneticAlgorithmParameters(
    MUTPB=0.15,
    INDPB=0.1,
    ETA=5,
    CXPB=0.7,
    NGEN=20,
    POP_SIZE=1000,
    cost_function_weight=(-0.5, -0.5),
)

Add a constraint to limit the total of energy transfert coefficient to 100%.


In [13]:
constraint_energy = constraint.ConstraintNoTransportEnergyCoefficient(
    parameters_name=["D1N1_energy_coefficient", "D2N1_energy_coefficient"],
    min_energy_coef_value=0,
    max_energy_coef_value=1,
)

Finaly, create the Genetic Algorithm.


In [14]:
client = Client()
genetic_algo = GeneticAlgorithm(
    cost_function=cost_function,
    parameter_genetic_algorithm=genetic_algo_parameters,
    constraint=[constraint_energy],
    client=client,
)

/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 51623 instead
  warnings.warn(


And watch the magic on the Dask dashboard :


In [15]:
genetic_algo.client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:51623/status,
Dashboard: http://127.0.0.1:51623/status,Workers: 4
Total threads: 12,Total memory: 48.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51624,Workers: 4
Dashboard: http://127.0.0.1:51623/status,Total threads: 12
Started: Just now,Total memory: 48.00 GiB
Comm: tcp://127.0.0.1:51636,Total threads: 3
Dashboard: http://127.0.0.1:51638/status,Memory: 12.00 GiB
Nanny: tcp://127.0.0.1:51627,


## Run the optimization


In [16]:
viewer = genetic_algo.optimize()

,mean,std,min,max,valid,from_previous_generation
generation,,,,,,
0,56.009411,136.818440,0.005291,1187.225819,1000,0.000
1,6.809588,40.001668,0.003771,804.491654,1000,0.256
2,4.541787,45.281828,0.002993,940.277469,1000,0.234
3,5.060383,48.817665,0.002993,796.406359,1000,0.242
4,6.505611,70.995404,0.003016,1194.531055,1000,0.241
5,10.356910,70.768110,0.001861,732.622725,1000,0.240
6,6.138871,62.964312,0.001549,1166.160081,1000,0.227
7,12.609680,90.459259,0.001312,889.944114,1000,0.263
8,14.061029,98.440311,0.001807,1011.162282,1000,0.234


Generations: 100%|██████████| 20/20 [28:57<00:00, 86.87s/it]


## Optimization statistics


In [17]:
viewer.hall_of_fame

D1N1_energy_coefficient  D1N1_tr_max  D1N1_tr_rate  \
generation individual                                                       
19         657                        0.027366    46.116402     -0.045755   
18         789                        0.027366    24.013173     -0.003632   
17         495                        0.027366    24.013173     -0.014174   
18         617                        0.027366    24.013173     -0.014174   
           170                        0.027366    24.013173     -0.013961   
...                                        ...          ...           ...   
11         233                        0.477356    66.000247     -0.013653   
10         62                         0.491675    66.000247     -0.013653   
6          145                        0.477356    15.934695     -0.017307   
0          362                        0.418671    65.641130     -0.246389   
4          568                        0.438948    50.525116     -0.246477   

                       D1N1_inv_lambda_max  D1N1_inv_lambda_rate  \
generation individual                                              
19         657                  170.863582             -0.082659   
18         789                  178.165785             -0.082659   
17         495                  178.165785             -0.082659   
18         617                  178.165785             -0.082659   
           170                  178.165785             -0.082659   
...                                    ...                   ...   
11         233                  199.986100             -0.012249   
10         62                   194.161996             -0.012249   
6          145                  199.986100             -0.012249   
0          362                  192.566342             -0.005058   
4          568                  181.540390             -0.003950   

                       D2N1_energy_coefficient  D2N1_tr_max  D2N1_tr_rate  \
generation individual                                                       
19         657                        0.311535    73.693127     -0.001607   
18         789                        0.338759    48.722899     -0.263023   
17         495                        0.306650    25.679523     -0.008846   
18         617                        0.306650    59.768036     -0.008846   
           170                        0.306650    25.679523     -0.008846   
...                                        ...          ...           ...   
11         233                        0.202152    73.693127     -0.003760   
10         62                         0.309847    97.388566     -0.196394   
6          145                        0.127392    73.693127     -0.003760   
0          362                        0.362336    55.350207     -0.117671   
4          568                        0.076845    19.321340     -0.120339   

                       D2N1_inv_lambda_max  D2N1_inv_lambda_rate      fitness  
generation individual                                                          
19         657                  178.397446             -0.295874     0.000675  
18         789                  128.655593             -0.299901     0.000683  
17         495                  144.395962             -0.288991     0.000693  
18         617                  163.521936             -0.288991     0.000702  
           170                  144.395962             -0.288991     0.000724  
...                                    ...                   ...          ...  
11         233                  104.856693             -0.282254  1141.275238  
10         62                   118.736121             -0.288180  1148.747899  
6          145                  183.965987             -0.273568  1166.160081  
0          362                  131.877265             -0.161552  1187.225819  
4          568                  121.839595             -0.193896  1194.531055  

[15299 rows x 11 columns]

In [18]:
viewer.fitness_evolution()

In [19]:
viewer.box_plot(3, nbest=1000)

In [20]:
groups = [
    ["D1N1_energy_coefficient", "D1N1_tr_rate", "D1N1_tr_max", "D1N1_inv_lambda_rate", "D1N1_inv_lambda_max"],
    ["D2N1_energy_coefficient", "D2N1_tr_rate", "D2N1_tr_max", "D2N1_inv_lambda_rate", "D2N1_inv_lambda_max"],
]
fig = viewer.parallel_coordinates(
    nbest=1000,
    unselected_opacity=0,
    parameter_groups=groups,
)

In [21]:
for trace in fig:
    display(trace)

In [ ]:
if SAVE:
    for i, trace in enumerate(fig):
        trace.write_html(f"Parallel_coordinates_{export_file_name}_{i}.html")

# Prepare results for ploting


## Run simulation with original parameters


In [23]:
original_config = [[0, 0, 0.1668, 10.38, -0.11, 150, -0.15]]
print(f"Original configuration: {list(original_config[0])}")

Original configuration: [0, 0, 0.1668, 10.38, -0.11, 150, -0.15]


In [24]:
original_model = wrapper.model_generator_no_transport(
    forcing_parameters=forcing_parameters,
    fg_parameters=wrapper.FunctionalGroupGeneratorNoTransport(parameters=original_config, groups_name=["Total"]),
)

original_model.run()

## Run simulation with optimized parameters


In [25]:
best_individual = cost_function.functional_groups.generate_matrix(viewer.hall_of_fame.iloc[0])
print(f"Best individual: {list(best_individual[0])}")
print(f"Best individual: {list(best_individual[1])}")

Best individual: [0.0, 0.0, 0.02736569821137916, 46.11640151336343, -0.04575452130273703, 170.86358205543152, -0.08265901699612513]
Best individual: [1.0, 0.0, 0.3115346079666376, 73.69312655089976, -0.0016069301923028602, 178.3974455696452, -0.29587362305691955]


In [26]:
best_model = wrapper.model_generator_no_transport(
    forcing_parameters=forcing_parameters,
    fg_parameters=wrapper.FunctionalGroupGeneratorNoTransport(parameters=best_individual, groups_name=["D1N1", "D2N1"]),
)

best_model.run()

And with the X best individuals :


In [27]:
NB_INDIVIDUAL_TO_SHOW = 10

min_biomass_D1N1 = (
    best_model.export_biomass().sel(latitude=[LATITUDE], longitude=[LONGITUDE]).sel(functional_group=0).data.flatten()
)
max_biomass_D1N1 = (
    best_model.export_biomass().sel(latitude=[LATITUDE], longitude=[LONGITUDE]).sel(functional_group=0).data.flatten()
)
min_biomass_D2N1 = (
    best_model.export_biomass().sel(latitude=[LATITUDE], longitude=[LONGITUDE]).sel(functional_group=1).data.flatten()
)
max_biomass_D2N1 = (
    best_model.export_biomass().sel(latitude=[LATITUDE], longitude=[LONGITUDE]).sel(functional_group=1).data.flatten()
)

biomass_accumulated = []

for cpt in range(0, NB_INDIVIDUAL_TO_SHOW):
    individual = cost_function.functional_groups.generate_matrix(viewer.hall_of_fame.iloc[cpt])
    model = wrapper.model_generator_no_transport(
        forcing_parameters=forcing_parameters,
        fg_parameters=wrapper.FunctionalGroupGeneratorNoTransport(parameters=individual, groups_name=["D1N1", "D2N1"]),
    )
    model.run()

    biomass_accumulated.append(
        model.export_biomass().sel(latitude=[LATITUDE], longitude=[LONGITUDE]).expand_dims({"individual": [cpt]})
    )

In [28]:
with xr.set_options(keep_attrs=True):
    best_individuals_simulations = xr.concat(biomass_accumulated, dim="individual")
    min_biomass_D1N1 = (
        best_individuals_simulations.sel(functional_group=0).min("individual").isel(latitude=0, longitude=0)
    )
    max_biomass_D1N1 = (
        best_individuals_simulations.sel(functional_group=0).max("individual").isel(latitude=0, longitude=0)
    )
    min_biomass_D2N1 = (
        best_individuals_simulations.sel(functional_group=1).min("individual").isel(latitude=0, longitude=0)
    )
    max_biomass_D2N1 = (
        best_individuals_simulations.sel(functional_group=1).max("individual").isel(latitude=0, longitude=0)
    )

### Format the results


Convert to pandas series.


In [29]:
optimized_biomass_pandas = (
    best_model.export_biomass()
    .sel(latitude=[LATITUDE], longitude=[LONGITUDE])
    .pint.quantify()
    .pint.to("mg/meter^2")
    .pint.dequantify()
    .to_dataframe()
    .pivot_table(index="time", columns="functional_group", values="biomass")
)
original_biomass_pandas = (
    original_model.export_biomass()
    .sel(latitude=[LATITUDE], longitude=[LONGITUDE])
    .pint.quantify()
    .pint.to("mg/meter^2")
    .pint.dequantify()
    .to_dataframe()
    .pivot_table(index="time", columns="functional_group", values="biomass")
)
observations_day_pandas = (
    observations_hot.pint.quantify()
    .pint.to("mg/meter^2")
    .pint.dequantify()
    .day.dropna("time")
    .to_dataframe()
    .reset_index()
    .set_index("time")["day"]
)
observations_night_pandas = (
    observations_hot.pint.quantify()
    .pint.to("mg/meter^2")
    .pint.dequantify()
    .night.dropna("time")
    .to_dataframe()
    .reset_index()
    .set_index("time")["night"]
)
layer_pandas = (
    epi_layer_depth.sel(latitude=[LATITUDE], longitude=[LONGITUDE])
    .pint.dequantify()
    .to_dataframe()
    .reset_index()
    .set_index("time")["pelagic_layer_depth"]
)

Then resample to month.


In [30]:
monthly_obs_day = observations_day_pandas.resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_obs_day.index = monthly_obs_day.index.to_period("M").to_timestamp()

monthly_obs_night = observations_night_pandas.resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_obs_night.index = monthly_obs_night.index.to_period("M").to_timestamp()

monthly_pred_d1n1 = optimized_biomass_pandas.iloc[:, 0].resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_pred_d1n1.index = monthly_pred_d1n1.index.to_period("M").to_timestamp()

monthly_pred_d2n1 = optimized_biomass_pandas.iloc[:, 1].resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_pred_d2n1.index = monthly_pred_d2n1.index.to_period("M").to_timestamp()

monthly_pred_orignal = original_biomass_pandas.iloc[:, 0].resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_pred_orignal.index = monthly_pred_orignal.index.to_period("M").to_timestamp()

monthly_layer = layer_pandas.resample("ME").mean()[TIME_START:TIME_END].dropna()
monthly_layer.index = monthly_layer.index.to_period("M").to_timestamp()

# Plot


### Correlation matrix of X best individuals


In [38]:
NB_INDIVIDUAL_CORRELATION = 5000

indiv_param = viewer.hall_of_fame.iloc[:NB_INDIVIDUAL_CORRELATION, :-1].to_numpy()
param_names = viewer.hall_of_fame.columns[:-1]

corr_matrix = np.corrcoef(indiv_param.T)
np.fill_diagonal(corr_matrix, np.nan)

fig = px.imshow(
    corr_matrix,
    text_auto=False,
    aspect="auto",
    color_continuous_scale=[[0, "blue"], [0.5, "white"], [1, "red"]],
    zmin=-1,
    zmax=1,
    x=param_names,
    y=param_names,
)
fig.update_layout(
    title="Correlation Matrix of Hall of Fame Parameters (Lower Triangle)",
    plot_bgcolor="rgba(0,0,0,0)",
    paper_bgcolor="rgba(0,0,0,0)",
    xaxis={"showgrid": False, "tickangle": -35},
    yaxis={"showgrid": False},
)

fig.show()

In [32]:
if SAVE:
    fig.write_html(f"Correlation_best_individuals_{export_file_name}.html")

### Daily with best individuals


In [36]:
x = best_model.export_biomass().sel(latitude=[LATITUDE], longitude=[LONGITUDE]).time.to_series()
x_rev = pd.concat([x, x[::-1]])

fig = go.Figure()

# D1N1 data
y_D1N1 = (
    best_model.export_biomass()
    .sel(latitude=[LATITUDE], longitude=[LONGITUDE])
    .sel(functional_group=0)
    .pint.quantify()
    .pint.to("mg/m2")
    .pint.dequantify()
    .data.flatten()
)
y_D1N1_upper = max_biomass_D1N1.pint.quantify().pint.to("milligram / meter ** 2")
y_D1N1_lower = min_biomass_D1N1.pint.quantify().pint.to("milligram / meter ** 2")[::-1]
y_D1N1_rev = np.concatenate([y_D1N1_upper, y_D1N1_lower])

# D2N1 data
y_D2N1 = (
    best_model.export_biomass()
    .sel(latitude=[LATITUDE], longitude=[LONGITUDE])
    .sel(functional_group=1)
    .pint.quantify()
    .pint.to("mg/m2")
    .pint.dequantify()
    .data.flatten()
) + y_D1N1
y_D2N1_upper = max_biomass_D2N1.pint.quantify().pint.to("milligram / meter ** 2") + y_D1N1_upper
y_D2N1_lower = min_biomass_D2N1.pint.quantify().pint.to("milligram / meter ** 2")[::-1] + y_D1N1_lower
y_D2N1_rev = np.concatenate([y_D2N1_upper, y_D2N1_lower])

fig = go.Figure()

# D1N1 traces
fig.add_trace(
    go.Scatter(
        x=x_rev,
        y=y_D1N1_rev,
        fill="toself",
        fillcolor="rgba(255,0,0,0.2)",
        line_color="rgba(255,255,255,0)",
        name=f"D1N1 : {NB_INDIVIDUAL_TO_SHOW} best individuals",
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=y_D1N1,
        line_color="rgb(255,0,0)",
        name="D1N1 : Best individual",
    )
)

# D2N1 traces
fig.add_trace(
    go.Scatter(
        x=x_rev,
        y=y_D2N1_rev,
        fill="toself",
        fillcolor="rgba(0,176,246,0.2)",
        line_color="rgba(255,255,255,0)",
        name=f"D2N1 : {NB_INDIVIDUAL_TO_SHOW} best individuals",
    )
)

fig.add_trace(
    go.Scatter(
        x=x,
        y=y_D2N1,
        line_color="rgb(0,176,246)",
        name="D2N1 : Best individual",
    )
)

# OBSERVATIONS ------------------------------------------------ #

fig.add_trace(
    go.Scatter(
        x=observations_hot.day.time.data.flatten(),
        y=observations_hot.day.pint.quantify().pint.to("mg/m2").pint.dequantify().data.flatten(),
        mode="lines",
        name="Day observations",
        line={"dash": "dash", "color": "firebrick", "width": 1},
    )
)

fig.add_trace(
    go.Scatter(
        x=observations_hot.night.time.data.flatten(),
        y=observations_hot.night.pint.quantify().pint.to("mg/m2").pint.dequantify().data.flatten(),
        mode="lines",
        name="Night observations",
        line={"dash": "dash", "color": "royalblue", "width": 1},
    )
)

# PRIMARY PRODUCTION ----------------------------------------- #

fig.add_trace(
    go.Scatter(
        x=best_model.state.sel(latitude=[LATITUDE], longitude=[LONGITUDE]).primary_production.time.data.flatten(),
        y=best_model.state.sel(latitude=[LATITUDE], longitude=[LONGITUDE])
        .primary_production.pint.quantify()
        .pint.to("mg/m2/day")
        .pint.dequantify()
        .data.flatten(),
        mode="lines",
        name="Primary Production (mg/m2/day)",
        line={"dash": "dot", "color": "green", "width": 1},
    )
)

fig.update_layout(
    xaxis_title="Time",
    yaxis_title="Biomass (mg/m2)",
    title=f"Biomass over Time (Daily) : {export_file_name}",
    showlegend=True,
)

fig.show()

In [37]:
if SAVE:
    fig.write_html(f"Biomass_best_individuals_{export_file_name}.html")

### Monthly with original parameters


In [ ]:
fig = go.Figure(
    layout={
        "title": "SeapoPym Monthly Biomass during day (CAFE npp) and Observations : HOT station + all param. optimization + 2 groups",
        "xaxis": {"title": "Time"},
        "yaxis": {"title": "Biomass . Epipelagic layer depth (mg/m^2)"},
    }
)

# DAY ------------------------------------------------ #

fig.add_trace(
    go.Scatter(
        x=monthly_pred_d1n1.index,
        y=monthly_pred_d1n1,
        mode="lines",
        line={"color": "rgb(255,0,0)"},
        name="Predictions Day",
    )
)
fig.add_trace(
    go.Scatter(
        x=monthly_obs_day.index,
        y=monthly_obs_day,
        fill="tonextx",
        fillcolor="rgba(255,0,0,0.15)",
        mode="lines+markers",
        line={"color": "firebrick", "dash": "dash", "width": 1},
        name="Observations (day)",
    )
)
# Night ------------------------------------------------ #
fig.add_trace(
    go.Scatter(
        x=(monthly_pred_d1n1 + monthly_pred_d2n1).index,
        y=(monthly_pred_d1n1 + monthly_pred_d2n1),
        mode="lines",
        line={"color": "rgb(0,176,246)"},
        name="Predictions Night",
    )
)
fig.add_trace(
    go.Scatter(
        x=monthly_obs_night.index,
        y=monthly_obs_night,
        fill="tonextx",
        fillcolor="rgba(0,176,246,0.15)",
        mode="lines+markers",
        line={"color": "royalblue", "dash": "dash", "width": 1},
        name="Observations (night)",
    )
)

# LMTL ------------------------------------------------ #
fig.add_trace(
    go.Scatter(
        x=monthly_pred_orignal.index,
        y=monthly_pred_orignal,
        mode="lines",
        line={"color": "grey"},
        name="Predictions Original",
    )
)


# LAYER ------------------------------------------------ #

fig.add_trace(px.line(monthly_layer).data[0])

fig.update_traces(
    yaxis="y2",
    selector={"name": "pelagic_layer_depth"},
    line={"color": "green", "width": 1, "dash": "dot"},
)
fig.update_layout(
    yaxis2={"title": "Layer Depth (m)", "overlaying": "y", "side": "right"},
    height=600,
)
fig.update_layout(legend={"y": 1.0, "x": 1.1})
fig.show()

In [35]:
if SAVE:
    fig.write_html(f"Biomass_monthly_with_original_{export_file_name}.html")

### Taylor Diagram


In [ ]:
diagram = ModTaylorDiagram()

all_model = [monthly_pred_d1n1, monthly_pred_d1n1 + monthly_pred_d2n1, monthly_pred_orignal, monthly_pred_orignal]
all_obs = [monthly_obs_day, monthly_obs_night, monthly_obs_day, monthly_obs_night]

all_names = ["HOT Day", "HOT Night", "Original Day", "Original Night"]

for model, obs, name in zip(all_model, all_obs, all_names):
    diagram = generate_mod_taylor_diagram(diagram, obs=obs, model=model[obs.index], name=name)
diagram.plot()
plt.title(
    "Taylor Diagram for Seapodym model during day at HOT station with CAFE NPP : all parameters optimization and 2 groups"
)

# export the figure
if SAVE:
    plt.savefig(f"Taylor_{export_file_name}.png")

plt.show()

In [ ]:
if SAVE:
    diagram.get_stats().to_csv(f"Stats_{export_file_name}.csv", index=False)
diagram.get_stats()

### Telagrand diagram


In [38]:
# Quantify and convert observations to mg/m2
obs_day = (
    observations_hot.day.isel(latitude=0, longitude=0, layer=0).pint.quantify().pint.to("mg/m2").pint.dequantify()
    # .resample(time="1D")
    # .interpolate("linear")
)

# Select and convert predictions to mg/m2
pred_day = (
    best_individuals_simulations.isel(latitude=0, longitude=0, functional_group=0)
    .sel(time=obs_day.time)
    .pint.quantify()
    .pint.to("mg/m2")
    .pint.dequantify()
)

# Calculate ranks for day observations
all_ranks_day = []
for time_ite in obs_day.time:
    pred_day_ite = np.sort(pred_day.sel(time=time_ite).data)
    obs_day_ite = float(obs_day.sel(time=time_ite))
    rank = np.searchsorted(pred_day_ite, obs_day_ite)
    all_ranks_day.append(rank)

# Quantify and convert night observations to mg/m2
obs_night = (
    observations_hot.night.isel(latitude=0, longitude=0, layer=0).pint.quantify().pint.to("mg/m2").pint.dequantify()
    # .resample(time="1D")
    # .interpolate("linear")
)

# Select and convert night predictions to mg/m2
pred_night = pred_day + (
    best_individuals_simulations.isel(latitude=0, longitude=0, functional_group=1)
    .sel(time=obs_night.time)
    .pint.quantify()
    .pint.to("mg/m2")
    .pint.dequantify()
)

# Calculate ranks for night observations
all_ranks_night = []
for time_ite in obs_night.time:
    pred_night_ite = np.sort(pred_night.sel(time=time_ite).data)
    obs_night_ite = float(obs_night.sel(time=time_ite))
    rank = np.searchsorted(pred_night_ite, obs_night_ite)
    all_ranks_night.append(rank)

In [ ]:
fig = go.Figure()


value, count = np.unique(all_ranks_day, return_counts=True)
count = count / count.sum()
fig.add_trace(
    go.Bar(
        x=value,
        y=count,
        name="Day Ranks",
        marker_color="firebrick",
        opacity=0.75,
    )
)

value, count = np.unique(all_ranks_night, return_counts=True)
count = count / count.sum()
fig.add_trace(
    go.Bar(
        x=value,
        y=count,
        name="Night Ranks",
        marker_color="royalblue",
        opacity=0.75,
    )
)

# Update layout
fig.update_layout(
    title="Histogram of Ranks for Day and Night",
    xaxis_title="Rank",
    yaxis_title="P(Rank)",
    barmode="group",
)

fig.show()

In [40]:
fig.write_html(f"Telagrand_diagram_{export_file_name}.html")